In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


# 데이터 로드
situation = pd.read_csv('situation.csv', encoding='utf-8-sig')
situationCategory = pd.read_csv('situationCategory.csv', encoding='utf-8-sig')
situationKeyword = pd.read_csv('situationKeyword.csv', encoding='utf-8-sig')
goods = pd.read_csv('goods.csv', encoding='utf-8-sig')
goodsKeyword = pd.read_csv('goodsKeyword.csv', encoding='utf-8-sig')

# 그룹화 및 데이터 병합
situationKeyword = situationKeyword.groupby('situationKey')['situationKeyword'].apply(list).reset_index()
situation = pd.merge(situation, situationCategory, on='situationCateKey', how='inner')
situation = pd.merge(situation, situationKeyword, on='situationKey', how='inner')

goodsKeyword = goodsKeyword.groupby('ASIN')['goodsKeyword'].apply(list).reset_index()
goods = goods.merge(goodsKeyword, on='ASIN', how='inner')

# '탕비실' 관련 데이터 필터링
pantry_situation = situation[situation['situationCategory1'] == '탕비실']
pantry_goods = goods[goods['category1'] == '식음료']
situation = situation.drop(pantry_situation.index)


In [2]:
situation.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47 entries, 16 to 62
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   situationKey        47 non-null     int64 
 1   situationCateKey    47 non-null     int64 
 2   headline1           47 non-null     object
 3   headline2           47 non-null     object
 4   mainKeyword         47 non-null     object
 5   situationCategory1  47 non-null     object
 6   situationCategory2  47 non-null     object
 7   situationCategory3  47 non-null     object
 8   situationKeyword    47 non-null     object
dtypes: int64(2), object(7)
memory usage: 3.4+ KB


In [3]:
goods.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5061 entries, 0 to 5060
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   goodsKey         5061 non-null   int64  
 1   category1        5061 non-null   object 
 2   category2        5061 non-null   object 
 3   category3        5060 non-null   object 
 4   ASIN             5061 non-null   object 
 5   goodsName        5061 non-null   object 
 6   brand            4602 non-null   object 
 7   originalPrice    5061 non-null   int64  
 8   discountedPrice  5061 non-null   int64  
 9   ratingAvg        5061 non-null   float64
 10  ratingCount      5061 non-null   int64  
 11  goodsInfo        4859 non-null   object 
 12  goodsDesc        3106 non-null   object 
 13  goodsImg         5061 non-null   object 
 14  goodsKeyword     5061 non-null   object 
dtypes: float64(1), int64(4), object(10)
memory usage: 593.2+ KB


In [4]:
pantry_situation.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16 entries, 0 to 15
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   situationKey        16 non-null     int64 
 1   situationCateKey    16 non-null     int64 
 2   headline1           16 non-null     object
 3   headline2           16 non-null     object
 4   mainKeyword         16 non-null     object
 5   situationCategory1  16 non-null     object
 6   situationCategory2  16 non-null     object
 7   situationCategory3  16 non-null     object
 8   situationKeyword    16 non-null     object
dtypes: int64(2), object(7)
memory usage: 1.2+ KB


In [5]:
pantry_goods.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1057 entries, 2726 to 3782
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   goodsKey         1057 non-null   int64  
 1   category1        1057 non-null   object 
 2   category2        1057 non-null   object 
 3   category3        1057 non-null   object 
 4   ASIN             1057 non-null   object 
 5   goodsName        1057 non-null   object 
 6   brand            988 non-null    object 
 7   originalPrice    1057 non-null   int64  
 8   discountedPrice  1057 non-null   int64  
 9   ratingAvg        1057 non-null   float64
 10  ratingCount      1057 non-null   int64  
 11  goodsInfo        1049 non-null   object 
 12  goodsDesc        678 non-null    object 
 13  goodsImg         1057 non-null   object 
 14  goodsKeyword     1057 non-null   object 
dtypes: float64(1), int64(4), object(10)
memory usage: 132.1+ KB


In [6]:
def situation_recommendation(situation_df, goods_df):
    """일반 상황에 대한 상품 추천 (중복 방지 추가, goodsKey 사용)"""
    
    # TF-IDF 벡터라이저 초기화
    tfidf = TfidfVectorizer()
    
    # 무작위로 2개의 상황 샘플링
    random_situations = situation_df.sample(n=2)
    
    # 추천 결과를 저장할 딕셔너리
    recommendations = {}
    
    # 각 상황에 대해 추천 상품을 계산
    for _, row in random_situations.iterrows():
        # 상황의 headline1과 headline2 및 키워드 텍스트 설정
        headline1, headline2 = row['headline1'], row['headline2']
        situation_keywords = ' '.join(row['situationKeyword'])

        # 상품 데이터 복사 및 키워드 스트링으로 변환
        goods_copy = goods_df.copy()
        goods_copy['keyword_str'] = goods_copy['goodsKeyword'].apply(lambda x: ' '.join(x) if isinstance(x, list) else x)

        # 벡터화 및 유사도 계산
        # 상황 키워드를 벡터화하여 matrix로 변환
        situation_matrix = tfidf.fit_transform([situation_keywords])
        # 각 상품 키워드를 벡터화하여 matrix로 변환
        goods_matrix = tfidf.transform(goods_copy['keyword_str'])
        # 상황과 상품 간의 유사도를 계산하여 1차원 배열로 변환
        similarity_scores = cosine_similarity(goods_matrix, situation_matrix).flatten()
        
        # 상품 데이터에 유사도 점수 추가
        goods_copy['similarity'] = similarity_scores
        # 유사도 상위 50개 상품 선택
        top_50_goods = goods_copy.sort_values(by='similarity', ascending=False).head(50)
        
        # 상위 상품 중 카테고리별로 상위 6개의 카테고리 추출
        top_categories = (
            top_50_goods.groupby(['category1', 'category2', 'category3'])
            .size()
            .reset_index(name='count')
            .sort_values(by='count', ascending=False)
            .head(6)
        )
        
        # 카테고리별 추천 상품 목록 및 이미 선택된 상품의 키를 저장할 집합
        category_recommendations = []
        selected_goods_keys = set()  # 이미 선택된 상품의 goodsKey를 저장

        # 각 카테고리에 대해 추천 상품을 선택
        for _, category_row in top_categories.iterrows():
            category1, category2, category3 = category_row[['category1', 'category2', 'category3']]
            # 현재 카테고리의 상품 필터링
            filtered_goods = top_50_goods[
                (top_50_goods['category1'] == category1) &
                (top_50_goods['category2'] == category2) &
                (top_50_goods['category3'] == category3)
            ].sort_values(by='similarity', ascending=False)

            # 이미 선택된 상품 제거
            filtered_goods = filtered_goods[~filtered_goods['goodsKey'].isin(selected_goods_keys)]

            # 카테고리 내에서 최대 15개까지 고려
            filtered_goods = filtered_goods.head(15)

            # 추출할 상품 수 계산 (최대 2개)
            num_goods_to_sample = min(2, len(filtered_goods))
            
            # 추출 가능한 상품이 있을 경우 샘플링하여 추가
            if num_goods_to_sample > 0:
                sampled_goods = filtered_goods.sample(n=num_goods_to_sample, random_state=42)
                selected_goods_keys.update(sampled_goods['goodsKey'])
                category_recommendations.append(sampled_goods)

        # 모든 카테고리별 추천 상품을 하나의 DataFrame으로 병합
        all_recommendations = pd.concat(category_recommendations, ignore_index=True)
        
        # 총 추천 상품 수가 12개 미만일 경우 추가 상품 선택
        if len(all_recommendations) < 12:
            # 추가로 필요한 상품의 수 계산
            remaining_goods = top_50_goods[~top_50_goods['goodsKey'].isin(selected_goods_keys)]
            additional_goods_needed = 12 - len(all_recommendations)
            additional_goods = remaining_goods.head(additional_goods_needed)
            # 추가 상품을 병합하여 최종 추천 목록에 포함
            all_recommendations = pd.concat([all_recommendations, additional_goods], ignore_index=True)
            selected_goods_keys.update(additional_goods['goodsKey'])

        # 최종 추천 목록을 유사도 순으로 정렬 후 상위 12개 선택
        final_recommendations = all_recommendations.sort_values(by='similarity', ascending=False).head(12)
        # 상황 headline1과 headline2를 키로 하고 추천 상품을 값으로 추가
        recommendations[f"{headline1}, {headline2}"] = final_recommendations

    # 추천 결과 반환
    return recommendations


In [7]:
def pantry_situation_recommendation(pantry_situation, pantry_goods):
    """'탕비실' 상황에 대한 상품 추천 (중복 방지 추가, goodsKey 사용)"""

    # TF-IDF 벡터라이저 초기화
    tfidf = TfidfVectorizer()  # 독립적인 TF-IDF 벡터라이저 초기화
    
    # 무작위로 2개의 상황 샘플링
    random_situations = pantry_situation.sample(n=2)
    
    # 추천 결과를 저장할 딕셔너리
    recommendations = {}

    # 각 상황에 대해 추천 상품을 계산
    for _, row in random_situations.iterrows():
        # 상황의 headline1과 headline2 및 키워드 텍스트 설정
        headline1, headline2 = row['headline1'], row['headline2']
        situation_keywords = ' '.join(row['situationKeyword'])

        # 상품 데이터 복사 및 키워드 스트링으로 변환
        pantry_goods_copy = pantry_goods.copy()
        pantry_goods_copy['keyword_str'] = pantry_goods_copy['goodsKeyword'].apply(
            lambda x: ' '.join(x) if isinstance(x, list) else x
        )

        # 벡터화 및 유사도 계산
        # 상황 키워드를 벡터화하여 matrix로 변환
        situation_matrix = tfidf.fit_transform([situation_keywords])
        # 각 상품 키워드를 벡터화하여 matrix로 변환
        goods_matrix = tfidf.transform(pantry_goods_copy['keyword_str'])
        # 상황과 상품 간의 유사도를 계산하여 1차원 배열로 변환
        similarity_scores = cosine_similarity(goods_matrix, situation_matrix).flatten()

        # 상품 데이터에 유사도 점수 추가
        pantry_goods_copy['similarity'] = similarity_scores
        # 유사도 상위 50개 상품 선택
        top_50_goods = pantry_goods_copy.sort_values(by='similarity', ascending=False).head(50)
        
        # 상위 상품 중 카테고리별로 상위 3개의 카테고리 추출
        top_categories = (
            top_50_goods.groupby(['category1', 'category2', 'category3'])
            .size()
            .reset_index(name='count')
            .sort_values(by='count', ascending=False)
            .head(3)
        )

        # 카테고리별 추천 상품 목록 및 이미 선택된 상품의 키를 저장할 집합
        category_recommendations = []
        selected_goods_keys = set()  # 이미 선택된 상품의 goodsKey를 저장

        # 각 카테고리에 대해 추천 상품을 선택
        for _, category_row in top_categories.iterrows():
            category1, category2, category3 = category_row[['category1', 'category2', 'category3']]
            # 현재 카테고리의 상품 필터링
            filtered_goods = top_50_goods[
                (top_50_goods['category1'] == category1) &
                (top_50_goods['category2'] == category2) &
                (top_50_goods['category3'] == category3)
            ].sort_values(by='similarity', ascending=False)

            # 이미 선택된 상품 제거
            filtered_goods = filtered_goods[~filtered_goods['goodsKey'].isin(selected_goods_keys)]

            # 카테고리 내에서 최대 15개까지 고려
            filtered_goods = filtered_goods.head(15)

            # 추출할 상품 수 계산 (최대 4개)
            num_goods_to_sample = min(4, len(filtered_goods))

            # 추출 가능한 상품이 있을 경우 샘플링하여 추가
            if num_goods_to_sample > 0:
                sampled_goods = filtered_goods.sample(n=num_goods_to_sample, random_state=42)
                selected_goods_keys.update(sampled_goods['goodsKey'])
                category_recommendations.append(sampled_goods)

        # 모든 카테고리별 추천 상품을 하나의 DataFrame으로 병합
        all_recommendations = pd.concat(category_recommendations, ignore_index=True)

        # 총 추천 상품 수가 12개 미만일 경우 추가 상품 선택
        if len(all_recommendations) < 12:
            # 추가로 필요한 상품의 수 계산
            remaining_goods = top_50_goods[~top_50_goods['goodsKey'].isin(selected_goods_keys)]
            additional_goods_needed = 12 - len(all_recommendations)
            additional_goods = remaining_goods.head(additional_goods_needed)
            # 추가 상품을 병합하여 최종 추천 목록에 포함
            all_recommendations = pd.concat([all_recommendations, additional_goods], ignore_index=True)
            selected_goods_keys.update(additional_goods['goodsKey'])

        # 최종 추천 목록을 유사도 순으로 정렬 후 상위 12개 선택
        final_recommendations = all_recommendations.sort_values(by='similarity', ascending=False).head(12)
        
        # 상황 headline1과 headline2를 키로 하고 추천 상품을 값으로 추가
        recommendations[f"{headline1}, {headline2}"] = final_recommendations

    # 추천 결과 반환
    return recommendations


In [8]:
# 함수 실행 예시
result1 = situation_recommendation(situation, goods)

# 결과 출력
for situation_desc, recommended_goods in result1.items():
    print(f"\n상황: {situation_desc}")
    print(recommended_goods[['category1', 'category2', 'category3', 'goodsName', 'originalPrice', 'similarity']])



상황: 신속하고 안정적인, 최적의 네트워크 업무환경을 위한 필수 아이템
   category1     category2         category3  \
0       전자제품       데이터 저장소        내장 하드 드라이브   
8       전자제품  네트워크 연결 스토리지              디바이스   
11  사무용 전자제품   프린터 잉크 및 토너           토너 카트리지   
1       전자제품       데이터 저장소        내장 하드 드라이브   
2       전자제품  네트워크 연결 스토리지        내장 하드 드라이브   
3       전자제품  네트워크 연결 스토리지        내장 하드 드라이브   
4       전자제품     헤드폰 및 이어폰          이어버드 헤드폰   
5       전자제품     헤드폰 및 이어폰          이어버드 헤드폰   
6       전자제품       데이터 저장소  내부 솔리드 스테이트 드라이브   
7       전자제품       데이터 저장소  내부 솔리드 스테이트 드라이브   
9       전자제품  네트워크 연결 스토리지              디바이스   
10  사무용 전자제품   프린터 잉크 및 토너           토너 카트리지   

                                            goodsName  originalPrice  \
0   Seagate 바라쿠다 3TB 내장 하드 드라이브 HDD - 3.5인치 SATA 6...         116340   
8                 Synology 8베이 랙스테이션 RS1221+ (디스크 없음)        1696625   
11  HP 206X 마젠타색 고수익 토너 카트리지 | HP Color LaserJet P...         153582   
1   Seagate 바라쿠다 프로 퍼포먼스 내장 하드 드라이브 SATA HDD 1

In [9]:
# 결과를 CSV로 저장하는 코드
all_recommendations = []

for situation_desc, recommended_goods in result1.items():
    # 상황 설명을 각 추천 항목에 추가
    recommended_goods['situation'] = situation_desc
    all_recommendations.append(recommended_goods[['situation', 'category1', 'category2', 'category3', 'goodsName', 'originalPrice', 'similarity']])

# 모든 추천 결과를 하나의 데이터프레임으로 결합
all_recommendations_df = pd.concat(all_recommendations, ignore_index=True)

# CSV 파일로 저장
all_recommendations_df.to_csv('situation_recommendations.csv', index=False, encoding='utf-8-sig')


In [10]:
# 함수 실행 예시
result2 = pantry_situation_recommendation(pantry_situation, pantry_goods)

# 결과 출력
for situation_desc, recommended_goods in result1.items():
    print(f"\n상황: {situation_desc}")
    print(recommended_goods[['category1', 'category2', 'category3', 'goodsName', 'originalPrice', 'similarity']])



상황: 신속하고 안정적인, 최적의 네트워크 업무환경을 위한 필수 아이템
   category1     category2         category3  \
0       전자제품       데이터 저장소        내장 하드 드라이브   
8       전자제품  네트워크 연결 스토리지              디바이스   
11  사무용 전자제품   프린터 잉크 및 토너           토너 카트리지   
1       전자제품       데이터 저장소        내장 하드 드라이브   
2       전자제품  네트워크 연결 스토리지        내장 하드 드라이브   
3       전자제품  네트워크 연결 스토리지        내장 하드 드라이브   
4       전자제품     헤드폰 및 이어폰          이어버드 헤드폰   
5       전자제품     헤드폰 및 이어폰          이어버드 헤드폰   
6       전자제품       데이터 저장소  내부 솔리드 스테이트 드라이브   
7       전자제품       데이터 저장소  내부 솔리드 스테이트 드라이브   
9       전자제품  네트워크 연결 스토리지              디바이스   
10  사무용 전자제품   프린터 잉크 및 토너           토너 카트리지   

                                            goodsName  originalPrice  \
0   Seagate 바라쿠다 3TB 내장 하드 드라이브 HDD - 3.5인치 SATA 6...         116340   
8                 Synology 8베이 랙스테이션 RS1221+ (디스크 없음)        1696625   
11  HP 206X 마젠타색 고수익 토너 카트리지 | HP Color LaserJet P...         153582   
1   Seagate 바라쿠다 프로 퍼포먼스 내장 하드 드라이브 SATA HDD 1

In [11]:
# pantry_situation_recommendation 결과를 CSV로 저장하는 코드
pantry_recommendations = []

for situation_desc, recommended_goods in result2.items():
    # 상황 설명을 각 추천 항목에 추가
    recommended_goods['situation'] = situation_desc
    pantry_recommendations.append(recommended_goods[['situation', 'category1', 'category2', 'category3', 'goodsName', 'originalPrice', 'similarity']])

# 모든 추천 결과를 하나의 데이터프레임으로 결합
pantry_recommendations_df = pd.concat(pantry_recommendations, ignore_index=True)

# CSV 파일로 저장
pantry_recommendations_df.to_csv('pantry_situation_recommendations.csv', index=False, encoding='utf-8-sig')


In [12]:
def evaluate_recommendations_multiple_runs(recommendation_func, situation_df, goods_df, num_runs=100):
    """
    추천 함수를 여러 번 실행하여 평가 지표의 평균을 계산.

    recommendation_func: 평가할 추천 함수
    situation_df: 상황 데이터프레임
    goods_df: 상품 데이터프레임
    num_runs: 평가를 위한 실행 횟수
    """
    results = {
        "average_similarity": [],
        "precision_at_k": [],
        "category_accuracy": [],
        "diversity": []
    }
    
    # Precision@K 설정
    k = 12
    
    for _ in range(num_runs):
        # 추천 함수 실행
        recommendations = recommendation_func(situation_df, goods_df)
        
        # 각 실행에 대한 평가 결과 저장
        for key, recommended_goods in recommendations.items():
            # 1. 유사성 기반 평가
            avg_similarity = recommended_goods['similarity'].mean()
            results['average_similarity'].append(avg_similarity)
            
            # 2. Precision@K
            top_k_goods = recommended_goods.iloc[:k]  # 상위 k개의 추천 상품 선택
            precision_k = (top_k_goods['similarity'] > 0).sum() / k
            results['precision_at_k'].append(precision_k)
            
            # 3. 범주 정확도 개선: situationKeyword와 goodsKeyword 비교
            headline1, headline2 = key.split(', ')
            situation_row = situation_df[
                (situation_df['headline1'] == headline1.strip()) & (situation_df['headline2'] == headline2.strip())
            ]
            
            if not situation_row.empty:
                # 상황의 키워드를 불러오기
                situation_keywords = situation_row.iloc[0]['situationKeyword']
                if isinstance(situation_keywords, str):
                    situation_keywords = situation_keywords.split()
                situation_keywords = set(situation_keywords)
                
                # 추천 상품에서 goodsKeyword와 situationKeyword 간의 일치도를 계산
                def keyword_match(goods_keywords):
                    if isinstance(goods_keywords, str):
                        goods_keywords = goods_keywords.split()
                    goods_keywords = set(goods_keywords)
                    return len(situation_keywords & goods_keywords) > 0  # 교집합이 있으면 True
                
                matched_keywords = recommended_goods['goodsKeyword'].apply(keyword_match)
                category_accuracy = matched_keywords.mean()  # 일치하는 상품 비율로 정확도 계산
                results['category_accuracy'].append(category_accuracy)
            
            # 4. 다양성 평가
            unique_categories = recommended_goods['category1'].nunique()
            diversity_score = unique_categories / len(recommended_goods)
            results['diversity'].append(diversity_score)
    
    # 각 지표의 평균 계산 및 소수점 조정
    evaluation_results = {
        "Average Similarity": round(sum(results['average_similarity']) / len(results['average_similarity']), 3),
        "Precision@K": round(sum(results['precision_at_k']) / len(results['precision_at_k']), 3),
        "Category Accuracy": round(sum(results['category_accuracy']) / len(results['category_accuracy']), 3),
        "Diversity Score": round(sum(results['diversity']) / len(results['diversity']), 3)
    }
    
    return evaluation_results


In [13]:
## 일반 상황에 대해 평가지표

num_runs = 100  # 원하는 실행 횟수 설정
evaluation = evaluate_recommendations_multiple_runs(
    recommendation_func=situation_recommendation,
    situation_df=situation,
    goods_df=goods,
    num_runs=num_runs
)
print(evaluation)

{'Average Similarity': 0.562, 'Precision@K': 0.985, 'Category Accuracy': 0.983, 'Diversity Score': 0.152}


In [14]:
## 팬트리에 대해 평가지표
num_runs = 100  # 원하는 실행 횟수 설정
evaluation = evaluate_recommendations_multiple_runs(
    recommendation_func=pantry_situation_recommendation,
    situation_df=pantry_situation,
    goods_df=pantry_goods,
    num_runs=num_runs
)
print(evaluation)

{'Average Similarity': 0.519, 'Precision@K': 0.966, 'Category Accuracy': 0.966, 'Diversity Score': 0.083}


In [15]:
#1 샘플 결과와 그에 대한 평가 지표값 동시 출력
def evaluate_recommendations(recommended_goods, situation_df):
    results = {
        "average_similarity": [],
        "precision_at_k": [],
        "category_accuracy": [],
        "diversity": []
    }
    
    # Precision@K 설정
    k = 12
    
    # 평가 수행
    for key, recommended_goods in result1.items():
        # 1. 유사성 기반 평가
        avg_similarity = recommended_goods['similarity'].mean()
        results['average_similarity'].append(avg_similarity)
        
        # 2. Precision@K
        top_k_goods = recommended_goods.iloc[:k]  # 상위 k개의 추천 상품 선택
        precision_k = (top_k_goods['similarity'] > 0).sum() / k
        results['precision_at_k'].append(precision_k)
        
        # 3. 범주 정확도 개선: situationKeyword와 goodsKeyword 비교
        headline1, headline2 = key.split(', ')
        situation_row = situation_df[
            (situation_df['headline1'] == headline1) & (situation_df['headline2'] == headline2)
        ]
        
        if not situation_row.empty:
            # 상황의 키워드를 불러오기
            situation_keywords = set(situation_row.iloc[0]['situationKeyword'])
            
            # 추천 상품에서 goodsKeyword와 situationKeyword 간의 일치도를 계산
            def keyword_match(goods_keywords):
                goods_keywords = set(goods_keywords)  # goodsKeyword를 집합으로 변환
                return len(situation_keywords & goods_keywords) > 0  # 교집합이 있으면 True
            
            matched_keywords = recommended_goods['goodsKeyword'].apply(keyword_match)
            category_accuracy = matched_keywords.mean()  # 일치하는 상품 비율로 정확도 계산
            results['category_accuracy'].append(category_accuracy)
        
        # 4. 다양성 평가
        unique_categories = recommended_goods['category1'].nunique()
        diversity_score = unique_categories / len(recommended_goods)
        results['diversity'].append(diversity_score)
    
    # 평균 계산 및 소수점 조정
    evaluation_results = {
        "Average Similarity": round(sum(results['average_similarity']) / len(results['average_similarity']), 3),
        "Precision@K": round(sum(results['precision_at_k']) / len(results['precision_at_k']), 3),
        "Category Accuracy": round(sum(results['category_accuracy']) / len(results['category_accuracy']), 3),
        "Diversity Score": round(sum(results['diversity']) / len(results['diversity']), 3)
    }
    
    return evaluation_results

# 함수 실행 예시
result1 = situation_recommendation(situation, goods)

# 각 추천 결과에 대한 평가 출력
for situation_desc, recommended_goods in result1.items():
    print(f"\n상황: {situation_desc}")
    print(recommended_goods[['category1', 'category2', 'category3', 'goodsName', 'originalPrice', 'similarity']])
    
    # 평가 수행
    evaluation = evaluate_recommendations({situation_desc: recommended_goods}, situation)
    print("평가 결과:", evaluation)


상황: 효율적인 회의를 위한, 직장인 사무용품 모음 
   category1   category2      category3  \
0       사무용품      파일링 제품  파일 재킷 및 파일 포켓   
1       사무용품      파일링 제품  파일 재킷 및 파일 포켓   
2       사무용품  클립, 집게 및 링             클립   
3       사무용품  클립, 집게 및 링             클립   
4       사무용품  클립, 집게 및 링             집게   
5       사무용품  클립, 집게 및 링             집게   
6       사무용품      파일링 제품          봉투 파일   
7       사무용품      파일링 제품          봉투 파일   
8       사무용품      파일링 제품        다용도 정리함   
9       사무용품      파일링 제품        다용도 정리함   
10      사무용품      파일링 제품         확장형 파일   
11      사무용품      파일링 제품         확장형 파일   

                                            goodsName  originalPrice  \
0   1InTheOffice File Jackets 레터 사이즈 5.1cm(2인치) 확장...          30456   
1   Blue Summit 파일 재킷, 노트 안감, 레터 사이즈, 5.1cm(2인치) 확...          30456   
2   BAZIC 바인더 클립 미디엄 32mm(1 1/4인치) 블랙, 종이 클램프 종이 클...           8296   
3   종이 클립 500개 다양한 사이즈, 스몰, 미디엄 및 라지(2.8cm(1.1인치),...           8850   
4           50세트 플라스틱 파일 패스너 2홀 페이퍼 패스너 바인더 클립, 혼